# Welcome to NameWeave - Manual Backpropagation

Let's rearrange our last code from <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/NameWeave%20(MLP)%20-%20Activations%2C%20Gradients%20%26%20Batch%20Normalization.ipynb">NameWeave (MLP) - Activations, Gradients & Batch Normalization</a> file, to come back to the part where we were at the end of the file.

In this lecture, we will not try to achieve a very low loss (We would get around the same loss that we had before) or dive into any complexifying architecture to improve our performance in the names prediction, rather we would be diving into the manual back propagation...

Why would we do this you ask?

Because, back in the days of around 2012, everybody used to write the backward pass manually, and you can *shoot yourself in the foot* if you don't know how to manually back propagate through, while debugging a neural network when you run into optimization issues and much more...

Also, back propagation is a <a href="https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b#:~:text=%3E%20The%20problem%20with%20Backpropagation%20is,them%20work%E2%80%9D%20on%20your%20data.">leaky abstraction</a>. Try to look into Andrej's beautiful medium post about back propagation.

In this notebook, we would divide the excercises into 4 parts:
1. Manual Back Propagation for each and every expression
2. Manual Back Propagation through fast cross_entropy (Combined Back Propagation)
3. Manual Back Propagation through batch normalization (Combined Back Propagation)
4. Putting all the code together for manual `loss.backward()`

**Typically we would use the general `loss.backward()` nowadays (which is PyTorch's auto-grad engine), and not everybody knows how to backpropagate through the neural networks manually, but we would learn to manually do it, just like we used to, back in the days...**

Such that, this meme becomes relatable:
![Manual Back Propagation Meme](ExplanationMedia/Images/ManualBackPropMeme.png)

# Installing Dependencies

In [1]:
!pip install torch
!pip install numpy
!pip install pandas
!pip install matplotlib

# Importing Libraries


In [2]:
import random
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\avhis\AppData\Local\Temp\ipykernel_10444\3082457058.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Loading Dataset

In [3]:
words = open("Datasets/Indian_Names.txt").read().splitlines()
words = [word.lower() for word in words]

In [4]:
len(words)

53982

# Building Vocabulary

In [5]:
# Remember we need our starting and ending tokens as well in these mappings,
characters = sorted(list(set(''.join(words)))) # Gives us all the characters in the english alphabet, hopefully our dataset has all of them
stoi = {s:i+1 for i,s in enumerate(characters)} # Enumerate returns the tuples of number and string, which can then be mapped to string:index
# We manually add these tokens for convenience
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()} # After we have the string:index mapping, we can easily iterate over their items to map index:string
print("Characters:", characters)
print("STOI:", stoi)
print("ITOS", itos)
# We define a common vocabulary size
vocabularySize = len(stoi)
print("Vocabulary Size:", vocabularySize)

Characters: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
STOI: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
ITOS {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Vocabulary Size: 27


# Building Dataset

In [6]:
# We define a Block Size based on the number of characters we feed are going to feed to predict the next one
blockSize = 3
# Will build the dataset on only the words we take as input
def buildDataset(words):
    # We define two lists, inputs & outputs, where inputs are our blocks of the block size mentioned above and outputs are the label indexes
    inputs , outputs = [], []
    # We iterate over each word
    for word in words:
        # We define the block for each iteration and fill it with 0 values -> [0, 0, 0]
        block = [0] * blockSize
        # We run another loop for each word's character, here word also needs the ending token '.'
        for character in word + '.':
            # We take out the index from our look-up table
            index = stoi[character]
            # We append the input with our block
            inputs.append(block)
            # We append the output label with out index of the character
            outputs.append([index])
            # We then take the block, crop it 1 size from the left and append the next index to it (sliding window of name)
            block = block[1:] + [index]
    # We also convert these inputs and outputs to tensors for neural network processing
    inputs = torch.tensor(inputs)
    outputs = torch.flatten(torch.tensor(outputs))
    # We return the inputs and outputs
    return inputs, outputs

# We define a manual seed to random
random.seed(69)
# We shuffle all the words, so that the model receives all kinds of data
random.shuffle(words)
# We define two number of inputs
# We take the number of examples to 80% in the first variable
numberOfInputs1 = int(0.8*len(words))
# We take the number of examples to 90% in the first variable
numberOfInputs2 = int(0.9*len(words))
# Inputs and outputs that go till 80% of the examples
trainingInputs, trainingOutputs = buildDataset(words[:numberOfInputs1])
# Inputs and outputs that start at 80% of the examples and go till 90% of the examples
validationInputs, validationOutputs = buildDataset(words[numberOfInputs1:numberOfInputs2])
# Inputs and outputs that start at 90% of the examples
testInputs, testOutputs = buildDataset(words[numberOfInputs2:])

# We can check the numbers
print("Total Examples:",len(words)) # 100%
print("Training Examples:",len(words[:numberOfInputs1])) # 80%
print("Validation Examples:",len(words[numberOfInputs1:numberOfInputs2])) # 10%
print("Test Examples:",len(words[numberOfInputs2:])) # 10%

Total Examples: 53982
Training Examples: 43185
Validation Examples: 5398
Test Examples: 5399


# Utility Function to Compare Gradients

We would also use a new utility function in this lecture to compare the gradients in this lecture.\
The utility function tells us if we have an exact match, an approximate match and the maximum difference we have in our gradients.

In [7]:
def compare_gradients(description, manual_gradient, torch_tensor):
    """
    Compare manual gradients with PyTorch gradients.

    Parameters:
    - description: A string describing the comparison.
    - manual_gradient: The manually computed gradient.
    - torch_tensor: The PyTorch tensor for which gradients are computed.

    Prints the comparison results including exact match, approximate match, and maximum difference.
    """
    exact_match = torch.all(manual_gradient == torch_tensor.grad).item()
    approximate_match = torch.allclose(manual_gradient, torch_tensor.grad)
    max_difference = (manual_gradient - torch_tensor.grad).abs().max().item()
    print(f'{description:15s} | Exact: {str(exact_match):5s} | Approximate: {str(approximate_match):5s} | MaxDiff: {max_difference}')

# Neural Network Initialization - Weights, Biases & Parameters

You will see that I changed the initialization for the parameters to be small numbers...\
Normally you would set the biases to be all zeros, but I did set the biases to be small numbers...\
Sometimes if your variables are initialized at exactly zero, it can mask an incorrect implementation of a gradient...

You will also notice, unlike last time, we are still using the biases for the First Hidden Layer, despite of the batch normalization which is just for fun (we are going to have a gradient with respect to it and we can check wheter we are calculating it correctly or not...

In [8]:
# We will define a generator to give the same result on your machine, as of my machine
generator = torch.Generator().manual_seed(6942069420)
# Embedding Matrix (Input Layer)
embeddingFeatureSpaceLength = 10 
embeddingLookUpMatrix = torch.randn((vocabularySize, embeddingFeatureSpaceLength), generator=generator)
# Hidden Layer
numberOfHiddenLayerNeurons = 64
weightsOfHiddenLayer = torch.randn((blockSize*embeddingFeatureSpaceLength), numberOfHiddenLayerNeurons, generator=generator) * (5/3) / ((embeddingFeatureSpaceLength*blockSize)**0.5)
biasesOfHiddenLayer = torch.randn(numberOfHiddenLayerNeurons, generator=generator) * 0.1
# Output Layer / Final Layer
weightsOfFinalLayer = torch.randn(numberOfHiddenLayerNeurons, vocabularySize, generator=generator) * 0.1
biasesOfFinalLayer = torch.randn(vocabularySize, generator=generator) * 0.1
# Batch Normalization Layer
batchGains = torch.randn((1, numberOfHiddenLayerNeurons)) * 0.1 + 1.0
batchBiases = torch.randn((1, numberOfHiddenLayerNeurons)) * 0.1
# Parameters
parameters = [embeddingLookUpMatrix, weightsOfHiddenLayer, biasesOfHiddenLayer, weightsOfFinalLayer, biasesOfFinalLayer, batchGains, batchBiases]
# We set all the requires gradient to True
for parameter in parameters:
    parameter.requires_grad = True

# Forward Pass

You will see that we will significantly expand the forward pass because we have some lines of code like the 
```python
loss = F.cross_entropy(logits, trainingOutputs[indexes])
```
which is very much not understandable of what works under the hood, so I tried to bring back some of the explicit implementation of the loss function that we have inside the PyTorch's `cross_entropy` method.

Also, we will be running the forward pass for only 1 epoch for the time being, until we figure out all the back propagation elements, and we will be prepending a 'd' in front of the variables to make them understandable as backward pass variables in the context of $\frac{d}{dx}y$ equations.

Note: This back propagation is going to be way different than what we did back in our original <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/Neural%20Network%20with%20Derivatives.ipynb">Neural Network with Derivatives</a> notebook, because in that notebook we had single (individual) scalars, but here we have entire layers of neurons and we want to backpropagate through the entirity of them.

In [9]:
batchSize = 32
indexes = torch.randint(low=0, high=trainingInputs.shape[0], size=(batchSize,))
inputBatch, outputBatch = trainingInputs[indexes], trainingOutputs[indexes]
# Forward Pass Start
embedding = embeddingLookUpMatrix[inputBatch]
concatenatedEmbedding = embedding.view(embedding.shape[0], -1)
# Linear Layer - 1
hiddenLayerPreBatchNormStates = concatenatedEmbedding @ weightsOfHiddenLayer + biasesOfHiddenLayer
# Batch Normalization Layer
batchMeanAtIteration = 1/batchSize*hiddenLayerPreBatchNormStates.sum(0, keepdim=True)
batchDifference = hiddenLayerPreBatchNormStates - batchMeanAtIteration
batchDifferencePow2 = batchDifference**2
batchVariance = 1/(batchSize - 1)*(batchDifferencePow2).sum(0, keepdim=True) # Note: Bessel's Correction (which is division by n-1 instead of n)
batchVarianceInverse = (batchVariance + 1e-5)**-0.5
batchRawValue = batchDifference * batchVarianceInverse
hiddenLayerPreActivationStates = batchGains * batchRawValue + batchBiases
# Non-Linearity Layer
hiddenLayerPostActivationStates = torch.tanh(hiddenLayerPreActivationStates)
# Linear Layer - 2
logits = hiddenLayerPostActivationStates @ weightsOfFinalLayer + biasesOfFinalLayer
# Cross Entropy Loss (Same as F.cross_entropy() method)
maxLogits = logits.max(1, keepdim=True).values
normalizedLogits = logits - maxLogits # Subtracting it for numerical stability
counts = normalizedLogits.exp()
sumCounts = counts.sum(1, keepdims=True)
sumCountsInverse = sumCounts**-1 # For some reason (1/sumCounts) does not output with exact back propagation
probabilities = counts * sumCountsInverse
logProbabilities = probabilities.log()
loss = -logProbabilities[range(batchSize), outputBatch].mean()

# Backward Pass (Mini-Batch)
for parameter in parameters:
    parameter.grad = None
# Rataining Gradient For Checking It Later
for item in [logProbabilities, probabilities, sumCountsInverse, sumCounts, counts, normalizedLogits, maxLogits, logits, 
             hiddenLayerPostActivationStates, hiddenLayerPreActivationStates, batchRawValue, batchVarianceInverse,
             batchVariance, batchDifferencePow2, batchDifference, batchMeanAtIteration, hiddenLayerPreBatchNormStates,
             concatenatedEmbedding, embedding]: # I am sorry, but I did not find a much cleaner way to do this XD
    item.retain_grad()
loss.backward()
print("Loss:", loss)

Loss: tensor(3.4479, grad_fn=<NegBackward0>)


# Manual Back Propagation

In this exercise we will back propagate through the entire forward pass for all the small expressions, one-by-one...

So let's start...

Let's understand what we are trying to achieve in the first place...

At the very beginning we are trying to calcualate the gradients of the loss...

In the context of sensitivity analysis, **gradients represent how sensitive one variable (output) is to small changes in another variable (input)**.

So, we are trying to know that **how sensitive the loss is in small changes to `logProbabilities`**...

Let's understand what $\log{}$ is first...

$\log{}\rightarrow\text{Inverse Function to Exponentiation}$

For example:\
$1000 = 10^3 \rightarrow \log_{10}(1000) \rightarrow 3$\
Or\
$x = b^y \rightarrow \log_{10}(x) \rightarrow y$

And we have natural logarithms as well... Also represented as $\ln{}$...\
Here we do $y=e^x\rightarrow\log_e{x}\rightarrow\ln{x}$.


Here we see that `e` is the base, and `e` is also known as the Euler's Number

$$ e = \sum\limits_{n=0}^\infty\frac{1}{n!} = \frac{1}{0!} + \frac{1}{1!} + \frac{1}{2!} + \frac{1}{3!} +...$$

So what is $\frac{d}{dx}(\log{(x)})$?

Well, $\frac{d}{dx}(\log{(x)}) = \frac{1}{x}$

Also if we remember $\log{}$ from our school days of mathematics, we know that $\log{(a.b.c)}=\log(a)+\log(b)+\log(c)$

Let's first understand what are we doing in this line:
```python
loss = -logProbabilities[range(batchSize), outputBatch].mean()
```

`range(batchSize)` gives us an array of `32` numbers from `0` to `31`, when we index into `logProbabilities` using `range()` and `outputBatch` range is essentially using all the `32` rows of the output and *plucking-out* all the **`logProbabilities` of the correct next character**, then it is averaging them using `mean()` and we are taking the `negetive` of that number.

So from all the above explanations we come to a simpler way of trying to evaluate something complex...\
Here in this line `loss = -logProbabilities[range(batchSize), outputBatch].mean()` we are doing something like:\
$$\text{loss} = -(a + b + c)/3$$
So if we do $\frac{dLoss}{da}$ we get:
$$loss = -\frac{1}{3}a + -\frac{1}{3}b + -\frac{1}{3}c$$\
Or $\frac{dLoss}{da} = -\frac{1}{3}$\
Or we can say that it is $-\frac{1}{n}$ of only those indexes who participate, otherwise all the other gradients remain 0.

But now we have to stay careful, because we understand that we don't have an individual scalar here...\
If we check the size of the `logProbabilities` using:
```python
print(logProbabilities.shape)
```
we get a shape like:
```python
torch.Size([32, 27])
```

And by this time, it should not surprise us, that the shape of the tensor remain the same because we are doing element-wise operations and assigning gradients to the same locations as the original tensor.

So now what we can do is, we can create a tensor of zeros and only fill the indexes with equation $-1/n$ where the indeces participate...

(For us `n` is the `batchSize`)

In [10]:
dLogProbabilities = torch.zeros_like(logProbabilities)
dLogProbabilities[range(batchSize), outputBatch] = -1.0/batchSize

# Now we can compare our backpropagation implementation
compare_gradients('dLogProbabilities', dLogProbabilities, logProbabilities)

dLogProbabilities | Exact: True  | Approximate: True  | MaxDiff: 0.0


Congratulations...\
We did our very first step...

Let's do the next step now...

For 
```python
logProbabilities = probabilities.log()
```

Once again,
$$\frac{d}{dx}\log{(x)} = \frac{1}{x}$$

But now, because this is an intermediate node, we have to be careful for local and global derivatives as well...

So, we will have $1/\text{Probailities}$ as the **local derivative** and our already caculated `dLogProbabilities` as our global derivative...

In [11]:
dLogProbabilities = torch.zeros_like(logProbabilities)
dLogProbabilities[range(batchSize), outputBatch] = -1.0/batchSize
dProbabilities = (1/probabilities) * dLogProbabilities


# Now we can compare our backpropagation implementation
compare_gradients('dLogProbabilities', dLogProbabilities, logProbabilities)
compare_gradients('dProbabilities', dProbabilities, probabilities)

dLogProbabilities | Exact: True  | Approximate: True  | MaxDiff: 0.0
dProbabilities  | Exact: True  | Approximate: True  | MaxDiff: 0.0


Seems like, we are very well on track... 😊

In [12]:
print(maxLogits.shape)
print(normalizedLogits.shape)
print(counts.shape)
print(sumCounts.shape)
print(sumCountsInverse.shape)
print(probabilities.shape)
print(logProbabilities.shape)

torch.Size([32, 1])
torch.Size([32, 27])
torch.Size([32, 27])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 27])
torch.Size([32, 27])
